In [ ]:
# Imports e paths

from pathlib import Path
import re
import pandas as pd
import numpy as np
from datetime import datetime
import random
import os

# Paths relativos ao local deste notebook: notebooks/bertopic/01_prep_light.ipynb
BASE = Path("../../data")
RAW = BASE / "raw"
OUT_DIR = BASE / "interim" / "bertopic"
IN_CSV = RAW / "tccs.csv"
OUT_CSV = OUT_DIR / "prep.csv"

# Reprodutibilidade
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Garantir diretório de saída
OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"[{datetime.now()}] BASE={BASE.resolve()}")
print(f"IN_CSV={IN_CSV.resolve()}")
print(f"OUT_CSV={OUT_CSV.resolve()}")

[2025-08-27 16:13:22.887785] BASE=C:\Users\User\Desktop\TCC\Notebooks locais\analise_topicos_tcc\data
IN_CSV=C:\Users\User\Desktop\TCC\Notebooks locais\analise_topicos_tcc\data\raw\tccs.csv
OUT_CSV=C:\Users\User\Desktop\TCC\Notebooks locais\analise_topicos_tcc\data\interim\bertopic\prep.csv


In [ ]:
# Leitura e chaves

# Leitura
df = pd.read_csv(IN_CSV, encoding="utf-8")

# Validações mínimas
assert "resumo" in df.columns, "A coluna 'resumo' não foi encontrada em data/raw/tccs.csv"

# Garante DOC_ID sequencial se não existir
if "DOC_ID" not in df.columns:
    df.insert(0, "DOC_ID", range(len(df)))

# Snapshot inicial
print(f"Shape: {df.shape}")
print("Colunas:", list(df.columns)[:10], "...")
display(df.head(3))

In [ ]:
# Função clean_text_light

# Compilações de regex (URLs, e-mails, números isolados)
RE_URL = re.compile(r"https?://\S+|www\.\S+", flags=re.IGNORECASE)
RE_EMAIL = re.compile(r"\b\S+@\S+\b", flags=re.IGNORECASE)
RE_NUM_ISOLATED = re.compile(r"\b\d+\b")
RE_SPACES = re.compile(r"\s+")

def clean_text_light(s: str) -> str:
    """
    Pré-processamento leve para BERTopic:
      - lower() mantendo acentos;
      - remove URLs e e-mails;
      - remove números isolados (não remove tokens alfanuméricos);
      - normaliza espaços;
      - NÃO remove stopwords aqui; NÃO lematiza; NÃO aplica unidecode;
      - mantém pontuação/acentos (útil ao modelo multilíngue).
    """
    s = s.lower()
    s = RE_URL.sub(" ", s)
    s = RE_EMAIL.sub(" ", s)
    s = RE_NUM_ISOLATED.sub(" ", s)
    s = RE_SPACES.sub(" ", s).strip()
    return s

In [ ]:
# Aplicação e diagnósticos

# Garante dtype string e trata NaN
orig_resumo = df["resumo"].copy()
df["RESUMO_PREP_BERTOPIC"] = (
    df["resumo"].fillna("").astype(str).map(clean_text_light)
)

# Diagnósticos simples (não filtrantes)
n_total = len(df)
n_vazios_antes = (orig_resumo.fillna("").astype(str).str.strip() == "").sum()
n_vazios_depois = (df["RESUMO_PREP_BERTOPIC"].str.strip() == "").sum()

print(f"Documentos totais: {n_total}")
print(f"Resumos vazios (antes): {n_vazios_antes}")
print(f"Resumos vazios (depois): {n_vazios_depois}")

# Pequeno check de mudanças (amostra)
sample_idx = df.index[:3].tolist()
for i in sample_idx:
    print("\n--- DOC_ID", df.loc[i, "DOC_ID"], "---")
    print("[ANTES]", str(orig_resumo.loc[i])[:300])
    print("[DEPOIS]", df.loc[i, "RESUMO_PREP_BERTOPIC"][:300])

In [ ]:
# Salvar e sumário

# Grava o CSV completo (mesmo schema + RESUMO_PREP_BERTOPIC)
df.to_csv(OUT_CSV, index=False, encoding="utf-8")

# Confirmação
ok_col = "RESUMO_PREP_BERTOPIC" in df.columns
print("\n======== RESUMO ========")
print(f"Arquivo salvo em: {OUT_CSV.resolve()}")
print(f"Linhas: {len(df)}  |  Coluna RESUMO_PREP_BERTOPIC presente? {ok_col}")
print("Primeiros campos:", list(df.columns)[:12], "...")
display(df[["DOC_ID", "RESUMO_PREP_BERTOPIC"]].head(5))
